Apriori updates

In [32]:
import pandas as pd
import pickle as pkl

In [33]:
with open('apriori_df_test.pkl', 'rb') as f:
    apriori_prev_df = pkl.load(f) # deserialize using load()

In [22]:
# sample rec data
recommendations = [['4u5gjtu4iybjhr', 486261, 2.0], [1676, 495271, 3.5]]

In [23]:
formatted_recommendations = [{'user_id': item[0], 'data': {'recipe_id': item[1], 'rating': int(item[2])}} for item in recommendations]
print(formatted_recommendations)

[{'user_id': '4u5gjtu4iybjhr', 'data': {'recipe_id': 486261, 'rating': 2}}, {'user_id': 1676, 'data': {'recipe_id': 495271, 'rating': 3}}]


In [24]:
for recommendation in formatted_recommendations:
    user_id = recommendation['user_id']
    new_dict = recommendation['data']
    print(new_dict)

{'recipe_id': 486261, 'rating': 2}
{'recipe_id': 495271, 'rating': 3}


In [25]:
for recommendation in formatted_recommendations:

    user_id = recommendation['user_id']
    new_dict = recommendation['data']

    if user_id not in apriori_prev_df['user_id'].values:
        new_row = {'user_id': user_id, 'rating_dict': [new_dict]}
        print(new_row)

        apriori_prev_df = pd.concat([apriori_prev_df, pd.DataFrame([new_row])], ignore_index=True)

        
    
    else:
        # find row index for associated user_id
        row_index = apriori_prev_df.index[apriori_prev_df['user_id'] == user_id][0]

        #appen new_dict to rating_dict of associated user_id
        apriori_prev_df.at[row_index, 'rating_dict'].append(new_dict)

    # Print the updated rating_dict for the user_id
    print(apriori_prev_df[apriori_prev_df['user_id'] == user_id]['rating_dict'].iloc[0])

[{'recipe_id': 486261, 'rating': 2}, {'recipe_id': 486261, 'rating': 2}]
[{'recipe_id': 490418, 'rating': 5}, {'recipe_id': 495271, 'rating': 3}, {'recipe_id': 495271, 'rating': 3}]


In [ ]:
# update csv for collaborative filter training
apriori_prev_df.to_csv('apriori_df.csv')

In [27]:
recipe_listing = apriori_prev_df['rating_dict']
recipe_listing

0        [{'recipe_id': 494972, 'rating': 4}, {'recipe_...
1        [{'recipe_id': 490418, 'rating': 5}, {'recipe_...
2        [{'recipe_id': 476060, 'rating': 4}, {'recipe_...
3        [{'recipe_id': 477675, 'rating': 5}, {'recipe_...
4        [{'recipe_id': 500256, 'rating': 2}, {'recipe_...
                               ...                        
10260                 [{'recipe_id': 502353, 'rating': 5}]
10261                 [{'recipe_id': 491465, 'rating': 5}]
10262                 [{'recipe_id': 516662, 'rating': 5}]
10263                 [{'recipe_id': 491512, 'rating': 4}]
10264    [{'recipe_id': 486261, 'rating': 2}, {'recipe_...
Name: rating_dict, Length: 10265, dtype: object

In [28]:
transformed_recipe_listing = []
for i in recipe_listing:
    sub_list = []
    for j in i:

        # only include recipes rated above 3
        if j['rating'] >= 3:
            sub_list.append(j['recipe_id'])

    transformed_recipe_listing.append(sub_list)

transformed_recipe_listing

[[494972, 495227, 516391, 476476, 499705, 477446],
 [490418, 495271, 495271],
 [476060, 491229],
 [477675,
  478370,
  484765,
  486688,
  487467,
  487917,
  488646,
  490038,
  496288,
  498173,
  504347,
  480070,
  504948,
  508709,
  512488,
  511645,
  484017,
  512584,
  505539,
  504212,
  513329,
  514791,
  515167,
  516981,
  519463,
  520295],
 [500256, 529720],
 [497651, 502899, 507064, 501231, 506781, 506876, 499217, 500848, 513224],
 [490713, 516009],
 [531410],
 [481964],
 [475856,
  481332,
  487628,
  482833,
  484807,
  481474,
  487470,
  479255,
  509630,
  510563,
  510617,
  512051,
  513127,
  492492,
  518396,
  511000,
  515371,
  517143,
  486905],
 [494930],
 [490779,
  488734,
  492790,
  481231,
  484426,
  505083,
  485082,
  510905,
  522887,
  489087,
  486294,
  522745,
  482360,
  533118,
  536119,
  517594,
  503916],
 [486675],
 [510268],
 [497554],
 [482277, 486710, 494334, 480341],
 [485138, 485813, 484528, 490917, 491003, 477386, 492776],
 [48671

In [29]:
from apyori import apriori

# transactions, min support, min confidence, min lift parameters to get relevant rules
rules = apriori(transactions = transformed_recipe_listing, min_support = 50/9955, min_confidence = 0.2, min_lift = 3, min_length=1,max_length = 2)
results = list(rules)


In [30]:
def apriori_table(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs1        = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    return list(zip(lhs, rhs1, supports))
apriori_recipe_df = pd.DataFrame(apriori_table(results), columns = ['Base Product', 'Add Product', 'Support'])

In [31]:
apriori_recipe_df.sort_values(by='Support', ascending=False, inplace=True)
apriori_recipe_df

,Base Product,Add Product,Support
13,486261,486267,0.004384
258,495124,495577,0.003312
293,495275,495577,0.002923
309,495577,496573,0.002728
257,495124,495275,0.002728
...,...,...,...
237,494784,514423,0.001072
238,494784,518145,0.001072
240,494953,495577,0.001072
115,487669,496573,0.001072


In [20]:
apriori_recipe_df.to_csv('recipe_mix.csv')


In [21]:
# pickle apriori rule csv
with open('./apriori_rules.pkl', 'wb') as f:
    pkl.dump(apriori_recipe_df, f)